# Statystyka matematyczna - ćwiczenia laboratoryjne 2023/2024

Ten notatnik zalicza się do grupy zestawów zadań, na podstawie których odbywa się zaliczenie ćwiczeń i podlega zwrotowi do oceny w ustalonym na zajęciach terminie.

Uwagi i wytyczne ogólne dotyczące uzupełniania notatnika:
- Podczas wykonywania zadań należy korzystać wyłącznie z pakietów zaimportowanych na początku notatnika oraz z pakietów wchodzących w skład standardowej biblioteki Pythona, które można zaimportować samodzielnie we wskazanej komórce notatnika.
- Swoje rozwiązania należy wprowadzać wyłącznie w miejce następujących fragmentów kodu:<br/> `# YOUR CODE HERE`<br/> `raise NotImplementedError()`<br/> 
a odpowiedzi tekstowe w komórkach oznaczonych hasłem:<br/> 
`YOUR ANSWER HERE`<br/> 
Nie należy w żaden sposób modyfikować pozostałych fragmentów kodu oraz innych elementów notatnika, w szczególności dodawać lub usuwać komórek oraz zmieniać nazwy pliku.
- Otrzymywane wyniki i odpowiedzi mają być rezultatem wykonania napisanego kodu.
- Zadanie należy wykonać w taki sposób, aby podczas wykonywania kodu nie zostały wyświetlone żadne ostrzeżenia.
- Zawarte w notatniku automatyczne testy mają charakter poglądowy. Dotyczą one wybranych aspektów zadań i mają za zadanie wyłapać podstawowe błędy. Przejście wszystkich testów nie oznacza, że zadanie jest wykonane w całości poprawnie.

Uwagi i wytyczne ogólne dotyczące wizualizacji wyników:
- Wszystkie wykresy powinny być wykonane w jednolitym, przejrzystym i czytelnym stylu, posiadać odpowiednio dobrane proporcje i zakresy wartości osi.
- Wykresy oraz ich osie powinny mieć nadane tytuły. Jeżeli w obrębie figury znajduje się więcej niż jeden wykres to figura również powinna mieć nadany tytuł. 
- Figury powinny mieć ustawione białe tło, tak, aby niezależnie od ustawień notatnika wszystkie elementy wykresów były dobrze widoczne (domyślnie tło jest przeźroczyste co może powodować problemy w notatnikach z ustawionym ciemnym tłem).
- Rozmiar poziomy figur nie powinien przekraczać 20 cali.

Przed odesłaniem zestawu zadań do oceny proszę uzupełnić komórkę z danymi autorów rozwiązania (nazwa zespołu oraz imiona, nazwiska i numery indeksów członków zespołu) oraz upewnić się, że notatnik działa zgodnie z oczekiwaniami. W tym celu należy skorzystać z opcji **Restart Kernel and Run All Cells...** dostępnej na górnej belce notatnika pod symbolem $\blacktriangleright\blacktriangleright$.

Nazwa zespołu: 2.38 Członkowie: Roksana Jandura , Katarzyna Wesołwoska , Katarzyna Tokarczuk , Magdalena Pogorzelec 

---

# Zestaw zadań 8: Estymacja metodą bootstrap

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Miejsce do importu pakietów wchodzących w skład standardowej biblioteki Pythona oraz ustawienie opcji wykorzystywanych pakietów

### Dane do zadań

W celu wygenerowania danych wykorzystywanych w zawartych w notatniku zadaniach i komórkach testowych wykonaj obie poniższe komórki.

In [3]:
sample_1 = pd.DataFrame(data=sp.stats.norm.rvs(loc=5, scale=0.2, size=20, random_state=7), columns=["X"])
sample_1.head()

,X
0,5.338105
1,4.906813
2,5.006564
3,5.081503
4,4.842215


In [4]:
x = sp.stats.uniform.rvs(loc=-2, scale=10, size=25, random_state=34)
y = 2*x - 5 + sp.stats.norm.rvs(loc=0, scale=2, size=25, random_state=13)
sample_2 = pd.DataFrame(data=np.array([x, y]).T, columns=["X", "Y"])
sample_2.head()

,X,Y
0,-1.614383,-9.653548
1,5.801005,8.109542
2,-1.072962,-7.234931
3,4.328927,4.561478
4,-1.861092,-6.031980


### Zadanie 1: Estymacja bootstapowa parametrów próby losowej [8 pkt]

Przygotuj funkcję `bootstrap_parameter_estimation()`, która korzystając z metody bootstrap będzie dokonywała estymacji przedziałowej podstawowych parametrów próby losowej - wartości oczekiwanej, wariancji i odchylenia standardowego dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

W obliczeniach skorzystaj z nieobciążonych estymatorów wariancji i odchylenia standardowego.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`P należy do przedziału [X, Y] przy założeniu poziomu ufności 1-𝛼 = Z`

gdzie P (symbol lub nazwa estymowanego parametru), X, Y i Z są automatycznie uzupełniane przez funkcję, a X i Y dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

Uwagi do wykonania zadania:
 - Automatyczne testy zakładają losowanie prób wg następującego schematu - kolejne próby są losowane w pęli for, a funkcja losująca próbę (np. `pd.sample()`) w kolejnych losowaniach ma ustawioną wartość parametru odpowiadającego za ziarno generatora liczb losowych na wartość `random_state`, `random_state+1`, `random_state+2` itd. 
 - Celem zadania jest napisanie funkcji od podstaw, w rozwiązaniu nie należy korzystać z gotowych funkcji służących do estymacji bootstrapowej.

In [5]:
def bootstrap_parameter_estimation(data, parameter, number_of_samples, alpha, random_state):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca domyślny indeks i pojedynczą kolumnę "X" z wartościami próby losowej.
    parameter: str
        W zależności od estymowanego parametru przyjmuje wartość "mean", "var" lub "std".
    number_of_samples: int
        Liczba prób bootstrapowych.
    alpha: float
        Wartość określająca poziom istotności.
    random_state: int
        Ziarno generatora liczb pseudolosowych.
    
    Returns
    -------
    parameter_low: float
        Dolna granica wyliczonego przedziału ufności.
    parameter_high: float
        Górna granica wyliczonego przedziału ufności.
    """       
    estimates = [] #lista z mean,var,std dla kazdej próby bo0otstrap
    #pętla d losowania kolejnych prób
    for i in range(number_of_samples):
        #ziarno generatora liczb losowych zwieksza sie za kazda iteracją0
        sample = data.sample(n=len(data), replace=True, random_state=random_state + i)
        
        if parameter == "mean":
            est = sample['X'].mean()
        elif parameter == "var":
            est = sample['X'].var(ddof=1) #ddof=1 bo nieobciążony estymator wariancji
        elif parameter == "std":
            est = sample['X'].std(ddof=1) #ddof=1 bo nieobciążony estymator std
        estimates.append(est) #dodaje  obliczona wartosc do listy
    
    lower_percentile = 100 * (alpha / 2) #dolny procentyl
    upper_percentile = 100 * (1 - alpha / 2) #górny percentyl
    parameter_low = np.percentile(estimates, lower_percentile) #dolna granica
    parameter_high = np.percentile(estimates, upper_percentile) #górna granica
    
    Z = 1 - alpha #poziom ufności
    print(f"{parameter} należy do przedziału [{parameter_low:.4f}, {parameter_high:.4f}] przy założeniu poziomu ufności 1-α = {Z}")
    
    return parameter_low, parameter_high


In [6]:
# Komórka testowa
assert np.all(np.isclose(bootstrap_parameter_estimation(sample_1, "mean", 1000, 0.05, 34), (4.913578501079754, 5.058598650238277)))
assert np.all(np.isclose(bootstrap_parameter_estimation(sample_1, "var", 2000, 0.02, 31), (0.009780029754528753, 0.05012098474121563)))
assert np.all(np.isclose(bootstrap_parameter_estimation(sample_1, "std", 1500, 0.1, 39), (0.11848597559111698, 0.20690074715252046)))

mean należy do przedziału [4.9136, 5.0586] przy założeniu poziomu ufności 1-α = 0.95
var należy do przedziału [0.0098, 0.0501] przy założeniu poziomu ufności 1-α = 0.98
std należy do przedziału [0.1185, 0.2069] przy założeniu poziomu ufności 1-α = 0.9


### Zadanie 2: Estymacja bootstrapowa wartości współczynnika korelacji liniowej [4 pkt]

Przygotuj funkcję `bootstrap_correlation_estimation()`, która korzystając z metody bootstrap będzie dokonywała estymacji przedziałowej wartości współczynnika korelacji liniowej Pearsona dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`r należy do przedziału [X, Y] przy założeniu poziomu ufności 1-𝛼 = Z`

gdzie X, Y i Z są automatycznie uzupełniane przez funkcję, a X i Y dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

Uwagi do wykonania zadania:
 - Automatyczne testy zakładają losowanie prób wg następującego schematu - kolejne próby są losowane w pęli for, a funkcja losująca próbę (np. `pd.sample()`) w kolejnych losowaniach ma ustawioną wartość parametru odpowiadającego za ziarno generatora liczb losowych na wartość `random_state`, `random_state+1`, `random_state+2` itd. 
 - Celem zadania jest napisanie funkcji od podstaw, w rozwiązaniu nie należy korzystać z gotowych funkcji służących do estymacji bootstrapowej.

In [7]:
def bootstrap_correlation_estimation(data, number_of_samples, alpha, random_state):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca domyślny indeks i dwie kolumny "X" i "Y" z wynikami próby losowej.
    number_of_samples: int
        Liczba prób bootstrapowych.
    alpha: float
        Wartość określająca poziom istotności.
    random_state: int
        Ziarno generatora liczb pseudolosowych.
    
    Returns
    -------
    r_corr_low: float
        Dolna granica wyliczonego przedziału ufności.
    r_corr_high: float
        Górna granica wyliczonego przedziału ufności.
    """
    # YOUR CODE HERE
    x = data['X']
    y = data['Y']

    # współczynniki korelacji z prób bootstrapowych
    bootstrap_correlations = []

    for i in range(number_of_samples):
        bootstrap_sample = data.sample(n=len(data), replace=True, random_state=random_state + i)
        correlation, _ = sp.stats.pearsonr(bootstrap_sample['X'], bootstrap_sample['Y'])
        bootstrap_correlations.append(correlation)

    bootstrap_correlations = sorted(bootstrap_correlations)

    lower_index = 100 * (alpha / 2)
    upper_index = 100 * (1 - alpha / 2)

    r_corr_low = np.percentile(bootstrap_correlations, lower_index)
    r_corr_high =  np.percentile(bootstrap_correlations, upper_index)
    
    confidence_level = 1 - alpha

    print(f"r należy do przedziału [{r_corr_low:.4f}, {r_corr_high:.4f}] przy założeniu poziomu ufności 1-𝛼 = {confidence_level:.2f}")

    return r_corr_low, r_corr_high

In [8]:
# Komórka testowa
assert np.all(np.isclose(bootstrap_correlation_estimation(sample_2, 1200, 0.05, 54), (0.9292256084679922, 0.9816422271248535)))
assert np.all(np.isclose(bootstrap_correlation_estimation(sample_2, 1500, 0.02, 14), (0.9171033819425604, 0.9840550768251518)))

r należy do przedziału [0.9292, 0.9816] przy założeniu poziomu ufności 1-𝛼 = 0.95
r należy do przedziału [0.9171, 0.9841] przy założeniu poziomu ufności 1-𝛼 = 0.98


### Zadanie 3: Estymacja bootstrapowa współczynników równania regresji liniowej [6 pkt]

Przygotuj funkcję `bootstrap_linear_regression_coefficients_estimation()`, która korzystając z metody bootstrap będzie dokonywała estymacji przedziałowej wartości współczynników równania regresji liniowej dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`a należy do przedziału [X1, Y1] przy założeniu poziomu ufności 1-𝛼 = Z1` </br>
`b należy do przedziału [X2, Y2] przy założeniu poziomu ufności 1-𝛼 = Z2`

gdzie X1, X2, Y1, Y2, Z1 i Z2 są automatycznie uzupełniane przez funkcję, a X1, X2, Y1 i Y2 dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

Uwagi do wykonania zadania:
 - Automatyczne testy zakładają losowanie prób wg następującego schematu - kolejne próby są losowane w pęli for, a funkcja losująca próbę (np. `pd.sample()`) w kolejnych losowaniach ma ustawioną wartość parametru odpowiadającego za ziarno generatora liczb losowych na wartość `random_state`, `random_state+1`, `random_state+2` itd. 
 - Celem zadania jest napisanie funkcji od podstaw, w rozwiązaniu nie należy korzystać z gotowych funkcji służących do estymacji bootstrapowej.

In [9]:
def bootstrap_linear_regression_coefficients_estimation(data, number_of_samples, alpha, random_state):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca domyślny indeks i dwie kolumny "X" i "Y" z wynikami próby losowej.
    number_of_samples: int
        Liczba prób bootstrapowych.
    alpha: float
        Wartość określająca poziom istotności.
    random_state: int
        Ziarno generatora liczb pseudolosowych.
    
    Returns
    -------
    a_ci: tuple
        Zmienna typu tuple zawierajaca granice przedziału ufności parametru a (a_low, a_high).
    b_ci: tuple
        Zmienna typu tuple zawierajaca granice przedziału ufności parametru b (b_low, b_high).
    """            
    # YOUR CODE HERE
    #raise NotImplementedError()

    a_samples = []
    b_samples = []

    for i in range(number_of_samples):
        # Losuowanie próbki ze zwracaniem
        bootstrap_sample = data.sample(n=len(data), replace=True, random_state=random_state + i)
        
        # średnie X i Y
        mean_x = bootstrap_sample['X'].mean()
        mean_y = bootstrap_sample['Y'].mean()
        
        # współczynnik kierunkowy a
        licznik = np.sum((bootstrap_sample['X'] - mean_x) * (bootstrap_sample['Y'] - mean_y))
        mianownik = np.sum((bootstrap_sample['X'] - mean_x) ** 2)
        a = licznik / mianownik
        
        # stała b
        b = mean_y - a * mean_x
        
        a_samples.append(a)  # Współczynnik kierunkowy
        b_samples.append(b)  # Stała (przecięcie z osią Y)

    # Obliczanie przedziałów ufności
    a_low = np.percentile(a_samples, alpha * 100 / 2 )
    a_high = np.percentile(a_samples, 100 * (1 - alpha / 2))
    
    b_low = np.percentile(b_samples, alpha * 100 / 2 )
    b_high = np.percentile(b_samples, 100 * (1 - alpha / 2))

     # Poziom ufności
    confidence_level = 1 - alpha
    
    print(f"a należy do przedziału [{a_low:.4f}, {a_high:.4f}] przy założeniu poziomu ufności 1-𝛼 = {confidence_level}")
    print(f"b należy do przedziału [{b_low:.4f}, {b_high:.4f}] przy założeniu poziomu ufności 1-𝛼 = {confidence_level}")
   

    return (a_low, a_high), (b_low, b_high)

In [12]:
# Komórka testowa
assert np.all(np.isclose(bootstrap_linear_regression_coefficients_estimation(sample_2, 1800, 0.05, 71), ((1.8361393329086446, 2.2918023790827124), (-5.575502208944547, -3.6854174758043574))))
print()
assert np.all(np.isclose(bootstrap_linear_regression_coefficients_estimation(sample_2, 2100, 0.02, 62), ((1.7964929430496048, 2.337236979144576), (-5.771238137193083, -3.481203748551042))))

a należy do przedziału [1.8361, 2.2918] przy założeniu poziomu ufności 1-𝛼 = 0.95
b należy do przedziału [-5.5755, -3.6854] przy założeniu poziomu ufności 1-𝛼 = 0.95

a należy do przedziału [1.7965, 2.3372] przy założeniu poziomu ufności 1-𝛼 = 0.98
b należy do przedziału [-5.7712, -3.4812] przy założeniu poziomu ufności 1-𝛼 = 0.98


### Zadanie 4: Estymacja bootstrapowa za pomocą dedykowanych funkcji [7 pkt]

Korzystając z funkcji `scipy.stats.bootstrap()` dokonaj estymacji przedziałowej następujących parametrów na poziomie istotności 1-alpha = 0.98:
 - na podstawie zmiennej `sample_1`: średnia, wariancja, odchylenie standardowe,
 - na podstawie zmiannej `sample_2`: współczynnik korelacji liniowej Pearsona, współczynniki równania regresji liniowej a i b.
 
Wyniki zapisz do zmiennych o nazwach `mean_ci`, `var_ci`, `std_ci`, `r_ci`, `a_ci` oraz `b_ci` zawierających listy z dolnymi i górnymi granicami przedziałów ufności. Podczas obliczania wartości każdego z parametrów ustaw wartość argumentu funkcji `scipy.stats.bootstrap()` odpowiadającego za ziarno generatora liczb losowych na 25.

Uwagi do wykonania zadania:
 - Automatyczne testy przewidują obliczenie wartości wariancji i odchylenia standardowego dla wartości parametru `ddof=0`.

In [9]:
# mean
boot_strap = sp.stats.bootstrap((sample_1["X"],), statistic=np.mean, confidence_level=0.98, random_state=25)

mean_ci = [boot_strap.confidence_interval[0], boot_strap.confidence_interval[1]]
mean_ci

[4.890098551749027, 5.063500727815335]

In [10]:
# var
boot_strap = sp.stats.bootstrap((sample_1["X"],), statistic=np.var, confidence_level=0.98, random_state=25)

var_ci = [boot_strap.confidence_interval[0], boot_strap.confidence_interval[1]]
var_ci

[0.013001391902892368, 0.05670851692597062]

In [11]:
# std
boot_strap = sp.stats.bootstrap((sample_1["X"],), statistic=np.std, confidence_level=0.98, random_state=25)

std_ci = [boot_strap.confidence_interval[0], boot_strap.confidence_interval[1]]
std_ci

[0.11423521149909441, 0.23829111055240196]

In [16]:
# r
def corr_p(x, y):
    return sp.stats.pearsonr(x, y)[0]

boot_strap = sp.stats.bootstrap((sample_2["X"],sample_2["Y"]), statistic=corr_p, paired=True, confidence_level=0.98, random_state=25)
r_ci = [boot_strap.confidence_interval[0], boot_strap.confidence_interval[1]]
r_ci

[0.9156715777126913, 0.9822743695601275]

In [18]:
# a
def reg(x, y):
    regress= sp.stats.linregress(x, y)
    return regress[0], regress[1]
boot_strap = sp.stats.bootstrap((sample_2["X"],sample_2["Y"]), reg, paired=True, confidence_level=0.98, random_state=25)
a_ci = [boot_strap.confidence_interval[0][0], boot_strap.confidence_interval[1][0]]
a_ci

[1.792948341896073, 2.3282111601384945]

In [13]:
# b
def reg(x, y):
    regress= sp.stats.linregress(x, y)
    return regress[0], regress[1]
boot_strap = sp.stats.bootstrap((sample_2["X"],sample_2["Y"]), sp.stats.linregress, paired=True, confidence_level=0.98, random_state=25)
b_ci = [boot_strap.confidence_interval[0][1], boot_strap.confidence_interval[1][1]]
b_ci

[-5.700934784848322, -3.4292283266002834]

In [19]:
# Komórka testowa
assert np.all(np.isclose(mean_ci, (4.890098551749027, 5.063500727815335)))
assert np.all(np.isclose(var_ci, (0.013001391902892368, 0.05670851692597062)))
assert np.all(np.isclose(std_ci, (0.11423521149909441, 0.23829111055240196)))
assert np.all(np.isclose(r_ci, (0.9156715777126913, 0.9822743695601276)))
assert np.all(np.isclose(a_ci, (1.792948341896073, 2.3282111601384945)))
assert np.all(np.isclose(b_ci, (-5.700934784848322, -3.4292283266002834)))